In [25]:
import torch
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn
from IPython.display import Image

import glob
import random
import os
import uuid



# Постановка задачи

Имеются изображения технологического процесса разлива металлических циллиндров. Есть риск нарушения технологии, когда стенки цилиндра не успевают застывать и трескаются. Не застывшый металл выливается, не образуя требуюмую заготовку. Необходимо оперативно определить лунку где произошел пролив. При этом пролив дольно редкое явление. Гораздо больше изображений можно собрать без пролива в лунке.

Задача: построить модель (на основе автоэнкодера) определяющую состояние лунки пролив\не пролив.

![title](.\imgs\CAM_C_02-03-2023_01-41-38-850.jpg)



## Датесет

Данные - вырезанные изображения лунок.

```
dataset
├── proliv  # изображения с проливами
|       ├── 000.jpg
│       ├── 001.jpg
│       │   └── ...
|
├── test  # тестовая выборка где перемешаны проливы и не_проливы
│       ├── imgs
│       │   ├── 000.jpg
│       │   ├── 001.jpg
│       │   └── ...
│       └── test_annotation.txt
|
├── train  #  обучающая выборка из не_проливов
|       ├── 000.jpg
│       ├── 001.jpg
│       └── ...
```

## План рещения

1. Имплементировать или найти автоэкодер (можно для старта взять пример из лекции по автоэнкодерам)
2. Обучаем автоэнкодер на не_проливах (dataset\train)
3. Если через такой автоэнкодер прогнать изображение пролива, то MSE между входным изображением и выходным будет больше, чем если прогнать изображение без пролива. Следовательно, если определить некторое пороговое значение MSE, можно классифицировать изображение на классы пролив\не_пролив. Если MSE между входной картинкой и выходной больше фиксированного порога, то на изображении пролив.
4. В качестве loss функции используем MSE (как минимум для baseline)
5. Для определения порога используем изображения из dataset\proliv
6. Пишем метод классификации лунок
7. На изображениях из dataset\test тестируем качество. Считаем True_positive_rate и True_negative_rate.
8. Цель: получить на тесте максимизаровать метрики True_positive_rate и True_negative_rate (получить более 91% по каждой)